# producing database
# read files with no rows dropped, concat together 

In [1]:
import pandas as pd
import numpy as np
import geopandas
import matplotlib.pyplot as plt
from pandas.io.parsers import read_csv


In [2]:
sbg=pd.read_excel('half-way/'+'SBG'+'.xlsx')
sbg=sbg.loc[:,['year','PE','tech_type', 'Bezirk','Gemeinde','KG_NR','KG']]
sbg['BL']='Salzburg'
sbg.groupby('tech_type').PE.sum()

tech_type
BKF          4484.333333
Bel.         6904.266667
Fest         1080.000000
Kompost       124.133333
MBR           268.000000
PKA          5490.066667
Primary      4406.666667
RBC           598.333333
SBR          4640.466667
Tropf        1692.693333
Unbekannt     912.666667
Name: PE, dtype: float64

In [3]:
karn=pd.read_excel('half-way/'+'karn'+'.xlsx')
karn=karn.loc[:,['year','PE','tech_type', 'Bezirk','Gemeinde','KG_NR','KG']]
karn['BL']='Carintia'
karn.groupby('tech_type').PE.sum()


tech_type
BKF           2928.133333
Bel.         40017.213333
Fest           476.666667
PKA           5451.786667
Primary      40933.733333
RBC             84.666667
SBR           5762.733333
Tropf          178.000000
Unbekannt     5374.293151
Name: PE, dtype: float64

In [4]:
noe=pd.read_excel('half-way/'+'noe'+'.xlsx')
noe=noe[['INBETRIEBNAHME','EW60','tech_type','BEZIRK']]
noe.rename(columns={'INBETRIEBNAHME':'year','EW60':'PE','BEZIRK':'Bezirk' },inplace=True)
noe['BL']='Lower-Austria'
noe[noe.tech_type=='Unbekannt'].PE.sum()
noe.groupby('tech_type').PE.sum()

tech_type
0              400.0
Andere        2730.0
BKF           2624.0
Bel.         22772.0
Fest           324.0
Kompost          5.0
MBR             52.0
PKA          13338.0
Primary       5098.0
RBC            203.0
SBR          63423.0
Tropf         1641.0
Unbekannt    20034.0
Wirbel          26.0
Name: PE, dtype: float64

In [5]:
tirol=pd.read_excel('half-way/'+'tirol'+'.xlsx')
tirol=tirol[['year','PE','tech_type', 'Bezirk','Gemeinde','KG_NR','KG']]
tirol['BL']='Tirol'
tirol.groupby('tech_type').PE.sum()

tech_type
0            3372.666667
BKF          1029.500000
Bel.         7959.533333
Fest          213.000000
Kompost       461.333333
PKA          1857.966667
Primary     10321.200000
Rotation      444.000000
SBR          3933.026667
Tropf        2403.000000
Wirbel        390.000000
Name: PE, dtype: float64

In [6]:
steyr=pd.read_excel('half-way/'+'steyr'+'.xlsx')
steyr=steyr[['year','PE','tech_type', 'Gemeinde','KG_NR','KG']]
steyr['BL']='Stirya'
steyr.groupby('tech_type').PE.sum()

tech_type
0               49
Andere         125
BKF           3353
Bel.         24076
Fest          3031
MBR             10
PKA          26356
Primary      18683
SBR          10723
Tauch          367
Tropf         3653
Unbekannt      132
Name: PE, dtype: int64

In [7]:
oebo=pd.read_excel('half-way/'+'oebo'+'.xlsx')
oebo=oebo[['INBETRIEBNAHME','EW60','Verfahren', 'ANLAGENAME']]
oebo.rename(columns={'INBETRIEBNAHME':'year','EW60':'PE','Verfahren':'tech_type'},inplace=True)
oebo['BL']='Upper-Austria'
oebo.groupby('tech_type').PE.sum()

tech_type
Belebung      12593
Belebung +     1344
Filtration      180
PF             8799
SBR           15166
SBR +          5703
Sonstig        2237
mechanisch     2150
Name: PE, dtype: int64

In [8]:
#load minor states
BRG=pd.read_excel('half-way/complete/BRG.xlsx')
VRB=pd.read_excel('half-way/complete/VBG.xlsx')
VIE=pd.read_excel('half-way/complete/wien.xlsx')

In [9]:
files=[karn,steyr, noe,tirol,oebo,sbg, VIE,VRB,BRG]

In [10]:
data=pd.concat(files)

### remove mistkaes, unify categories


In [11]:
data.replace('Bewilligungzeitpunkt abgelaufen oder unbekannt',0,inplace=True)
data.replace('Bescheid mangelhaft',0,inplace=True)
data.replace('m³ pro Tag x 1000 / 150',0,inplace=True)
data.replace('[Liter pro Sekunde x 3600 (= 1 Std.) x 10] / 150',0,inplace=True)
data=data.fillna(0).astype({'PE':int})



In [35]:
data.tech_type=data.tech_type.replace(0, 'Unbekannt')
data.tech_type=data.tech_type.replace('Belebung', 'Bel.')
data.tech_type=data.tech_type.replace('Belebung +', 'Bel.')
data.tech_type=data.tech_type.replace('PF', 'PKA')
data.tech_type=data.tech_type.replace('Rotation', 'RBC')
data.tech_type=data.tech_type.replace('SBR +', 'SBR')
data.tech_type=data.tech_type.replace('Sonstig', 'Other')
data.tech_type=data.tech_type.replace('mechanisch', 'Primary')
data.tech_type=data.tech_type.replace('3-k', 'Primary')
data.tech_type=data.tech_type.replace('Filtersack', 'Primary')
data.tech_type=data.tech_type.replace('Kompost', 'Other')
data.tech_type=data.tech_type.replace('Filterkammer', 'Primary')



In [39]:
rep={'Kompost':'Other','Wirbel':'Other','0':'Unknown','Unbekannt':'Unknown','Andere':'Other'}

In [40]:
data.tech_type=data.tech_type.replace(rep)


In [41]:
data.groupby('tech_type').PE.sum()

tech_type
BKF            15236
Bel.          116052
Fest            5144
Filtration       180
MBR              330
Other          37862
PKA            62021
Primary        82256
RBC             1479
SBR           111572
Tauch            367
Tropf           9684
Name: PE, dtype: int32

In [14]:
data[data.BL=='Lower-Austria'].groupby('tech_type').PE.sum()

tech_type
Andere        2735
BKF           2624
Bel.         22772
Fest           324
MBR             52
PKA          13338
Primary       5098
RBC            203
SBR          63423
Tropf         1641
Unbekannt    20434
Wirbel          26
Name: PE, dtype: int32

In [42]:
data=data.astype({'year':int})

# BEzirk, Gemeinde and KG_NR need to be cleaned and unified. either number or string

In [26]:
data.to_excel('final/database.xlsx',index=False)

In [27]:
import pandas as pd
test=pd.read_excel('final/database.xlsx')
test.groupby('BL').count()

,year,PE,tech_type,Bezirk,Gemeinde,KG_NR,KG,ANLAGENAME,Adresse,ID Nr.
BL,,,,,,,,,,
Burgenland,23,23,23,23,23,23,23,23,23,23
Carintia,7276,7276,7276,7276,7276,7276,7276,7276,7276,7276
Lower-Austria,5029,5029,5029,5029,5029,5029,5029,5029,5029,5029
Salzburg,1732,1732,1732,1732,1732,1732,1732,1732,1732,1732
Stirya,10718,10718,10718,10718,10718,10718,10718,10718,10718,10718
Tirol,1294,1294,1294,1294,1294,1294,1294,1294,1294,1294
Upper-Austria,2633,2633,2633,2633,2633,2633,2633,2633,2633,2633
Vienna,25,25,25,25,25,25,25,25,25,25
Vorarlberg,156,156,156,156,156,156,156,156,156,156


In [28]:
test.groupby('BL').count()

,year,PE,tech_type,Bezirk,Gemeinde,KG_NR,KG,ANLAGENAME,Adresse,ID Nr.
BL,,,,,,,,,,
Burgenland,23,23,23,23,23,23,23,23,23,23
Carintia,7276,7276,7276,7276,7276,7276,7276,7276,7276,7276
Lower-Austria,5029,5029,5029,5029,5029,5029,5029,5029,5029,5029
Salzburg,1732,1732,1732,1732,1732,1732,1732,1732,1732,1732
Stirya,10718,10718,10718,10718,10718,10718,10718,10718,10718,10718
Tirol,1294,1294,1294,1294,1294,1294,1294,1294,1294,1294
Upper-Austria,2633,2633,2633,2633,2633,2633,2633,2633,2633,2633
Vienna,25,25,25,25,25,25,25,25,25,25
Vorarlberg,156,156,156,156,156,156,156,156,156,156


In [29]:
test.groupby('tech_type').count()

,year,PE,Bezirk,Gemeinde,KG_NR,KG,BL,ANLAGENAME,Adresse,ID Nr.
tech_type,,,,,,,,,,
0,25,25,25,25,25,25,25,25,25,25
Andere,253,253,253,253,253,253,253,253,253,253
BKF,1262,1262,1262,1262,1262,1262,1262,1262,1262,1262
Bel.,7331,7331,7331,7331,7331,7331,7331,7331,7331,7331
Fest,475,475,475,475,475,475,475,475,475,475
Filtration,24,24,24,24,24,24,24,24,24,24
MBR,27,27,27,27,27,27,27,27,27,27
PKA,5720,5720,5720,5720,5720,5720,5720,5720,5720,5720
Primary,6491,6491,6491,6491,6491,6491,6491,6491,6491,6491
